In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Multiply, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_dir = '/content/drive/MyDrive/Research Materials/Researches/Interpretability of Machine Learning Models in Medical Image Analysis A Study Using the RSNA Pneumonia Dataset/Dataset/Data'

In [ ]:
train_dir = 'D:\\BIOSE\\1010_XAI-Pneumonia\\Data'

In [ ]:
# Data Augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

# Training data generator
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

# Validation data generator
validation_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

In [ ]:
# Few-shot generator for training
def few_shot_batch_generator(generator, n_shots=5):
    while True:
        batch_images, batch_labels = [], []

        # Collect n_shots samples for the few-shot learning task
        for _ in range(n_shots):
            images, labels = next(generator)
            batch_images.append(images)
            batch_labels.append(labels)

        # Stack the images and labels
        batch_images = np.vstack(batch_images)
        batch_labels = np.hstack(batch_labels)


        batch_labels = batch_labels.reshape(-1, 1)

        yield batch_images, batch_labels


few_shot_train_gen = few_shot_batch_generator(train_generator, n_shots=5)

In [ ]:
# Squeeze-and-Excitation (SE) block
def se_block(input_tensor, ratio=16):
    channel_axis = -1  # For channels_last data format
    filters = input_tensor.shape[channel_axis]

    se_shape = (1, 1, filters)

    # Squeeze operation
    se = GlobalAveragePooling2D()(input_tensor)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    # Scale operation
    x = Multiply()([input_tensor, se])
    return x

In [ ]:
# Model Definition with SE block and dropout
def create_model_with_attention():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

    # Unfreeze more layers for fine-tuning
    for layer in base_model.layers[-100:]:
        layer.trainable = True

    # Use the base model's output
    x = base_model.output

    # Apply SE block
    x = se_block(x)

    # Global Average Pooling
    x = GlobalAveragePooling2D()(x)

    # Add Dropout to prevent overfitting
    x = Dropout(0.5)(x)

    # Final output layer for binary classification
    outputs = Dense(1, activation='sigmoid')(x)

    # Create the model
    model = Model(inputs=base_model.input, outputs=outputs)
    return model

In [ ]:
# Create the model
model = create_model_with_attention()

In [ ]:
# Compile the model with Adam optimizer and weight decay
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-6, decay=1e-5)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Training with early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

In [ ]:
# **************NEW:correct steps_per_epoch and validation_steps
steps_per_epoch = train_generator.samples // (5 * train_generator.batch_size)
validation_steps = validation_generator.samples // validation_generator.batch_size

# Train the model using few-shot learning
history = model.fit(
    few_shot_train_gen,
    validation_data=validation_generator,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    epochs=50,
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:
#Old: Train the model using few-shot learning
history = model.fit(
    few_shot_train_gen,
    validation_data=validation_generator,
    steps_per_epoch=len(train_generator) // 8,
    validation_steps=len(validation_generator),
    epochs=50,
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:
# Plot training and validation accuracy/loss
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

**Saving the Model**

In [ ]:
#Architecture + Weights
model.save('/content/drive/MyDrive/savedmodel/ResSENet50.keras')

In [1]:
#Class Labels
class_indices = train_generator.class_indices
np.save('/content/drive/MyDrive/savedmodel/class_indices.npy', class_indices)

In [2]:
#Preprocessing Parameters

preprocess_params = {"rescale": 1./255}
np.save('/content/drive/MyDrive/savedmodel/preprocess_params.npy', preprocess_params)